In [1]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from config import fetch_data

## **CHURN ANALYSIS**

### Churn Drivers Waterfall

In [2]:
churn_drivers_waterfall_query = "SELECT * FROM gold.analytics_churn_analysis ORDER BY at_risk_customers DESC"
churn_drivers_waterfall_df = fetch_data(churn_drivers_waterfall_query)

fig = go.Figure()
fig.add_trace(go.Bar(
    name="CLV at Risk",
    x=churn_drivers_waterfall_df["customer_segment"],
    y=churn_drivers_waterfall_df["total_clv_at_risk"],
    
    # Modern color with slight gradient feel
    marker=dict(
        color=churn_drivers_waterfall_df["avg_churn_risk_pct"],
        colorscale="Reds",
        showscale=True,
        colorbar=dict(
            title="Avg Churn Risk %",
            ticksuffix="%"
        )
    ),
    
    text=churn_drivers_waterfall_df["at_risk_customers"],
    texttemplate="%{text:,} customers",
    textposition="outside",
    
    hovertemplate=
        "<b>%{x}</b><br>" +
        "At-Risk Customers: %{text:,}<br>" +
        "CLV at Risk: <b>$%{y:,.2f}</b><br>" +
        "Avg Churn Risk: %{customdata[0]:.1f}%<extra></extra>",
    
    customdata=churn_drivers_waterfall_df[["avg_churn_risk_pct"]].values
))

fig.update_layout(
    title=dict(
        text="Customer Lifetime Value at Risk by Segment",
        x=0.01,
        font=dict(size=20, weight="bold")
    ),
    
    xaxis=dict(
        title="Customer Segment",
        tickangle=-30,
        showgrid=False
    ),
    
    yaxis=dict(
        title="Total CLV at Risk ($)",
        tickprefix="$",
        tickformat=",",
        showgrid=True,
        gridcolor="rgba(200,200,200,0.3)"
    ),
    
    height=520,
    
    plot_bgcolor="white",
    paper_bgcolor="white",
    
    margin=dict(t=80, l=80, r=40, b=100),
    
    showlegend=False
)

fig.show()

In [3]:
churn_drivers_waterfall_df

,customer_segment,at_risk_customers,avg_churn_risk_pct,avg_days_inactive,avg_accounts_per_customer,avg_closed_accounts,avg_recent_transactions,avg_transaction_size,avg_support_contacts,avg_negative_interactions,avg_unresolved_issues,avg_clv,total_clv_at_risk,last_updated
0,Affluent,3020,66.31,2601.8,3.0,0.8,0.1,169.26,0.5,0.2,0.2,50538.29,152625625.0,2025-12-27 10:04:32.142228+00:00
1,Business,3018,65.43,2613.8,3.0,0.8,0.1,185.81,0.5,0.2,0.2,50571.97,152626213.0,2025-12-27 10:04:32.142228+00:00
2,Premium,3004,66.69,2588.1,3.0,0.9,0.1,167.31,0.5,0.2,0.3,50470.11,151612197.0,2025-12-27 10:04:32.142228+00:00
3,Mass Market,2996,66.58,2556.1,3.0,0.8,0.1,179.30,0.5,0.2,0.3,49850.67,149352602.0,2025-12-27 10:04:32.142228+00:00


### Churn Indicators Heatmap

In [4]:
churn_indicators_heatmap_query = "SELECT * FROM gold.analytics_churn_analysis"
churn_indicators_heatmap_df = fetch_data(churn_indicators_heatmap_query)

metrics = [
    'avg_days_inactive',
    'avg_recent_transactions',
    'avg_unresolved_issues',
    'avg_negative_interactions',
    'avg_churn_risk_pct'
]

# Build Z matrix and Y labels
z_data = []
y_labels = []

label_map = {
    'avg_days_inactive': 'Days Inactive (Avg)',
    'avg_recent_transactions': 'Recent Transactions (Avg)',
    'avg_unresolved_issues': 'Unresolved Issues (Avg)',
    'avg_negative_interactions': 'Negative Interactions (Avg)',
    'avg_churn_risk_pct': 'Churn Risk (%)'
}

for metric in metrics:
    z_data.append(churn_indicators_heatmap_df[metric].values)
    y_labels.append(label_map[metric])

fig = go.Figure(
    data=go.Heatmap(
        z=z_data,
        x=churn_indicators_heatmap_df["customer_segment"],
        y=y_labels,
        
        colorscale="RdYlGn_r",  # Red = worse
        
        zmid=None,
        showscale=True,
        colorbar=dict(
            title="Risk Intensity",
            tickformat=".1f",
            thickness=14
        ),
        
        text=z_data,
        texttemplate="%{text:.1f}",
        textfont=dict(size=10),
        
        hovertemplate=
            "<b>%{y}</b><br>" +
            "Segment: %{x}<br>" +
            "Value: %{z:.2f}<extra></extra>"
    )
)

fig.update_layout(
    title=dict(
        text="Churn Risk Indicators by Customer Segment",
        x=0.01,
        font=dict(size=20, weight="bold")
    ),
    
    xaxis=dict(
        title="Customer Segment",
        tickangle=-30,
        showgrid=False
    ),
    
    yaxis=dict(
        title="Risk Indicator",
        showgrid=False
    ),
    
    height=520,
    
    plot_bgcolor="white",
    paper_bgcolor="white",
    
    margin=dict(t=80, l=140, r=40, b=100)
)

fig.show()

In [5]:
churn_indicators_heatmap_df

,customer_segment,at_risk_customers,avg_churn_risk_pct,avg_days_inactive,avg_accounts_per_customer,avg_closed_accounts,avg_recent_transactions,avg_transaction_size,avg_support_contacts,avg_negative_interactions,avg_unresolved_issues,avg_clv,total_clv_at_risk,last_updated
0,Affluent,3020,66.31,2601.8,3.0,0.8,0.1,169.26,0.5,0.2,0.2,50538.29,152625625.0,2025-12-27 10:04:32.142228+00:00
1,Business,3018,65.43,2613.8,3.0,0.8,0.1,185.81,0.5,0.2,0.2,50571.97,152626213.0,2025-12-27 10:04:32.142228+00:00
2,Premium,3004,66.69,2588.1,3.0,0.9,0.1,167.31,0.5,0.2,0.3,50470.11,151612197.0,2025-12-27 10:04:32.142228+00:00
3,Mass Market,2996,66.58,2556.1,3.0,0.8,0.1,179.30,0.5,0.2,0.3,49850.67,149352602.0,2025-12-27 10:04:32.142228+00:00


## **FRAUD PATTERN ANALYSIS**

### Fraud Pattern Bubble

In [6]:
fraud_pattern_bubble_query = "SELECT * FROM gold.analytics_fraud_pattern_analysis  WHERE total_transactions >= 100  ORDER BY fraud_rate_pct DESC LIMIT 50"
fraud_pattern_bubble_df = fetch_data(fraud_pattern_bubble_query)

median_fraud_rate = fraud_pattern_bubble_df["fraud_rate_pct"].median()
median_amount = fraud_pattern_bubble_df["avg_transaction_amount"].median()

fig = px.scatter(
    fraud_pattern_bubble_df,
    x="avg_transaction_amount",
    y="fraud_rate_pct",
    size="total_transactions",
    color="channel",
    hover_name="merchant_category",
    
    hover_data={
        "total_transactions": ":,",
        "fraud_transactions": ":,",
        "avg_transaction_amount": ":$,.2f",
        "fraud_rate_pct": ":.2f%",
        "channel": False
    },
    
    labels={
        "avg_transaction_amount": "Avg Transaction Amount ($)",
        "fraud_rate_pct": "Fraud Rate (%)",
        "total_transactions": "Transaction Volume"
    },
    
    title="Fraud Risk Landscape: Transaction Amount vs Fraud Rate",
    
    size_max=60,
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Add median reference lines (quadrants)
fig.add_hline(
    y=median_fraud_rate,
    line_dash="dot",
    line_color="gray",
    annotation_text="Median Fraud Rate",
    annotation_position="top left"
)

fig.add_vline(
    x=median_amount,
    line_dash="dot",
    line_color="gray",
    annotation_text="Median Transaction Amount",
    annotation_position="top right"
)

fig.update_layout(
    height=600,
    
    plot_bgcolor="white",
    paper_bgcolor="white",
    
    xaxis=dict(
        showgrid=True,
        gridcolor="rgba(200,200,200,0.3)",
        tickprefix="$",
        tickformat=","
        # Uncomment if amounts are heavily skewed
        # type="log"
    ),
    
    yaxis=dict(
        showgrid=True,
        gridcolor="rgba(200,200,200,0.3)",
        ticksuffix="%",
        tickformat=".1f"
    ),
    
    legend_title_text="Channel",
    
    margin=dict(t=80, l=80, r=40, b=60)
)

fig.show()

In [7]:
fraud_pattern_bubble_df

,merchant_category,channel,is_international,day_name,is_weekend,hour_of_day,total_transactions,fraud_transactions,fraud_rate_pct,total_fraud_amount,avg_fraud_score,avg_transaction_amount,avg_distance_from_home,avg_velocity_24h,unique_customers,fraud_customers,last_updated
0,Retail,POS,True,Sunday,True,0.0,183,3,1.64,3607.85,0.792,1520.52,244.5,5.4,181,3,2025-12-27 10:04:35.572132+00:00
1,Utilities,Mobile,False,Friday,False,0.0,147,2,1.36,566.45,0.554,1571.83,266.5,5.4,147,2,2025-12-27 10:04:35.572132+00:00
2,Travel,Online,True,Friday,False,0.0,152,2,1.32,4229.36,0.926,1584.44,257.2,5.3,152,2,2025-12-27 10:04:35.572132+00:00
3,Services,ATM,True,Saturday,True,0.0,160,2,1.25,2812.94,0.643,1556.17,241.4,5.5,159,2,2025-12-27 10:04:35.572132+00:00
4,Healthcare,Mobile,False,Thursday,False,0.0,165,2,1.21,1824.05,0.265,1710.71,272.8,5.5,163,2,2025-12-27 10:04:35.572132+00:00
5,Services,Mobile,False,Tuesday,False,0.0,166,2,1.20,4658.56,0.042,1698.93,246.1,5.3,165,2,2025-12-27 10:04:35.572132+00:00
6,Utilities,Branch,False,Wednesday,False,0.0,170,2,1.18,4494.39,0.723,1596.93,280.4,5.6,170,2,2025-12-27 10:04:35.572132+00:00
7,Gas Station,Branch,True,Saturday,True,0.0,177,2,1.13,2598.81,0.528,1721.56,253.6,5.5,177,2,2025-12-27 10:04:35.572132+00:00
8,Entertainment,ATM,False,Tuesday,False,0.0,112,1,0.89,1794.52,0.509,1702.66,257.8,5.8,112,1,2025-12-27 10:04:35.572132+00:00
9,Restaurant,Mobile,False,Friday,False,0.0,130,1,0.77,970.53,0.899,1308.85,246.5,6.0,128,1,2025-12-27 10:04:35.572132+00:00


### Fraud Time Heatmap

In [8]:
fraud_time_heatmap_query = "SELECT * FROM gold.analytics_fraud_pattern_analysis"
fraud_time_heatmap_df = fetch_data(fraud_time_heatmap_query)

# Create pivot table
pivot = fraud_time_heatmap_df.pivot_table(
    values='fraud_rate_pct',
    index='hour_of_day',
    columns='day_name',
    aggfunc='mean'
)

# Reorder days
day_order = ['Monday', 'Tuesday', 'Wednesday',
                'Thursday', 'Friday', 'Saturday', 'Sunday']
pivot = pivot.reindex(
    columns=[d for d in day_order if d in pivot.columns])

fig = go.Figure(data=go.Heatmap(
    z=pivot.values,
    x=pivot.columns,
    y=pivot.index,
    colorscale='Reds',
    text=pivot.values,
    texttemplate='%{text:.2f}%',
    textfont={"size": 9},
    hovertemplate='<b>%{x}</b><br>Hour: %{y}<br>Fraud Rate: %{z:.2f}%<extra></extra>'
))

fig.update_layout(
    title="Fraud Rate Heatmap by Day & Hour",
    xaxis_title="Day of Week",
    yaxis_title="Hour of Day",
    height=600
)
fig.show()

In [9]:
fraud_time_heatmap_df

,merchant_category,channel,is_international,day_name,is_weekend,hour_of_day,total_transactions,fraud_transactions,fraud_rate_pct,total_fraud_amount,avg_fraud_score,avg_transaction_amount,avg_distance_from_home,avg_velocity_24h,unique_customers,fraud_customers,last_updated
0,Retail,POS,True,Sunday,True,0.0,183,3,1.64,3607.85,0.792,1520.52,244.5,5.4,181,3,2025-12-27 10:04:35.572132+00:00
1,Utilities,Mobile,False,Friday,False,0.0,147,2,1.36,566.45,0.554,1571.83,266.5,5.4,147,2,2025-12-27 10:04:35.572132+00:00
2,Travel,Online,True,Friday,False,0.0,152,2,1.32,4229.36,0.926,1584.44,257.2,5.3,152,2,2025-12-27 10:04:35.572132+00:00
3,Services,ATM,True,Saturday,True,0.0,160,2,1.25,2812.94,0.643,1556.17,241.4,5.5,159,2,2025-12-27 10:04:35.572132+00:00
4,Healthcare,Mobile,False,Thursday,False,0.0,165,2,1.21,1824.05,0.265,1710.71,272.8,5.5,163,2,2025-12-27 10:04:35.572132+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Entertainment,POS,False,Wednesday,False,0.0,164,0,0.00,0.00,NaN,1596.22,234.6,5.4,163,0,2025-12-27 10:04:35.572132+00:00
96,Entertainment,POS,False,Tuesday,False,0.0,132,0,0.00,0.00,NaN,1656.79,256.6,6.0,132,0,2025-12-27 10:04:35.572132+00:00
97,Entertainment,Mobile,False,Saturday,True,0.0,182,0,0.00,0.00,NaN,1722.52,236.6,5.7,181,0,2025-12-27 10:04:35.572132+00:00
98,Entertainment,POS,False,Thursday,False,0.0,147,0,0.00,0.00,NaN,1580.18,221.0,5.2,147,0,2025-12-27 10:04:35.572132+00:00


## **LOAN DEFAULT ANALYSIS**

### Loan Default Drivers Bar

In [10]:
loan_default_drivers_bar_query = "SELECT * FROM gold.analytics_loan_default_drivers  ORDER BY avg_late_payment_rate DESC"
loan_default_drivers_bar_df = fetch_data(loan_default_drivers_bar_query)


loan_default_drivers_bar_df["segment_label"] = (
    loan_default_drivers_bar_df["customer_segment"] + " | " +
    loan_default_drivers_bar_df["credit_score_band"] + " | " +
    loan_default_drivers_bar_df["income_bracket"]
)

fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=loan_default_drivers_bar_df["avg_late_payment_rate"],
        y=loan_default_drivers_bar_df["segment_label"],
        orientation="h",
        
        marker=dict(
            color=loan_default_drivers_bar_df["avg_late_payment_rate"],
            colorscale="Reds",
            showscale=True,
            colorbar=dict(
                title="Late Payment %",
                tickformat=".1f",
                thickness=14
            )
        ),
        
        text=loan_default_drivers_bar_df["avg_late_payment_rate"],
        texttemplate="%{text:.1f}%",
        textposition="outside",
        
        hovertemplate=
            "<b>%{y}</b><br>" +
            "Late Payment Rate: <b>%{x:.2f}%</b><br>" +
            "Loan Count: %{customdata[0]:,}<br>" +
            "Avg Days Late: %{customdata[1]:.1f}<extra></extra>",
        
        customdata=loan_default_drivers_bar_df[
            ["loan_count", "avg_days_late"]
        ].values
    )
)

fig.update_layout(
    title=dict(
        text="Late Payment Risk Drivers by Customer Segment",
        x=0.01,
        font=dict(size=20, weight="bold")
    ),
    
    xaxis=dict(
        title="Late Payment Rate (%)",
        showgrid=True,
        gridcolor="rgba(200,200,200,0.3)",
        ticksuffix="%"
    ),
    
    yaxis=dict(
        title="",
        autorange="reversed",  # Highest risk at top
        showgrid=False
    ),
    
    height=720,
    
    plot_bgcolor="white",
    paper_bgcolor="white",
    
    margin=dict(t=80, l=320, r=40, b=60),
    
    showlegend=False
)

fig.show()

In [11]:
loan_default_drivers_bar_df

,customer_segment,credit_score_band,income_bracket,employment_status,risk_level,loan_count,avg_late_payment_rate,avg_days_late,avg_late_fees,avg_outstanding,accounts_with_late_payments,accounts_with_missed_payments,last_updated,segment_label
0,Business,Very Good,Medium,Self-Employed,Low Risk,2,61.54,7.5,77.43,181329.11,2,0,2025-12-27 10:04:35.586741+00:00,Business | Very Good | Medium
1,Affluent,Poor,Low,Unemployed,Medium Risk,1,50.00,5.1,283.33,167462.83,1,0,2025-12-27 10:04:35.586741+00:00,Affluent | Poor | Low
2,Premium,Good,Low,Unemployed,Medium Risk,1,45.83,3.6,397.18,143335.78,1,0,2025-12-27 10:04:35.586741+00:00,Premium | Good | Low
3,Mass Market,Very Good,Medium,Employed,Medium Risk,1,38.89,2.7,247.00,231919.18,1,0,2025-12-27 10:04:35.586741+00:00,Mass Market | Very Good | Medium
4,Affluent,Fair,Medium,Student,Medium Risk,1,36.36,3.2,147.29,59775.54,1,0,2025-12-27 10:04:35.586741+00:00,Affluent | Fair | Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,Affluent,Fair,Medium,Self-Employed,Low Risk,1,0.00,0.0,0.00,106832.55,0,0,2025-12-27 10:04:35.586741+00:00,Affluent | Fair | Medium
835,Mass Market,Very Good,Low,Student,Low Risk,1,0.00,0.0,0.00,305834.94,0,0,2025-12-27 10:04:35.586741+00:00,Mass Market | Very Good | Low
836,Premium,Poor,Low,Employed,Low Risk,2,0.00,0.0,0.00,184118.01,0,0,2025-12-27 10:04:35.586741+00:00,Premium | Poor | Low
837,Premium,Very Good,Medium,Self-Employed,Low Risk,1,0.00,0.0,0.00,142300.98,0,0,2025-12-27 10:04:35.586741+00:00,Premium | Very Good | Medium


### Loan Default Correlation Matrix

In [12]:
loan_default_correlation_matrix_query = "SELECT * FROM gold.analytics_loan_default_drivers WHERE loan_count >= 50"
loan_default_correlation_matrix_df = fetch_data(loan_default_correlation_matrix_query)

numeric_cols = [
    'avg_late_payment_rate', 
    'avg_days_late', 
    'avg_late_fees',
    'avg_outstanding', 
    'accounts_with_late_payments'
]

corr_matrix = loan_default_correlation_matrix_df[numeric_cols].corr()

# Human-readable labels
labels = [col.replace('avg_', '').replace('_', ' ').title() for col in numeric_cols]

fig = go.Figure(
    data=go.Heatmap(
        z=corr_matrix.values,
        x=labels,
        y=labels,
        colorscale='RdBu',
        zmid=0,
        reversescale=False,
        showscale=True,
        colorbar=dict(
            title="Correlation",
            tickformat=".2f",
            thickness=14
        ),
        text=corr_matrix.values,
        texttemplate="%{text:.2f}",
        textfont=dict(size=11),
        hovertemplate="%{x} vs %{y}<br>Correlation: %{z:.3f}<extra></extra>"
    )
)

fig.update_layout(
    title=dict(
        text="Correlation Matrix: Loan Default Risk Factors",
        x=0.01,
        font=dict(size=20, weight="bold")
    ),
    xaxis=dict(tickangle=-45),
    yaxis=dict(autorange="reversed"),
    height=600,
    plot_bgcolor="white",
    paper_bgcolor="white",
    margin=dict(t=100, l=80, r=40, b=80)
)

fig.show()

In [13]:
loan_default_correlation_matrix_df

,customer_segment,credit_score_band,income_bracket,employment_status,risk_level,loan_count,avg_late_payment_rate,avg_days_late,avg_late_fees,avg_outstanding,accounts_with_late_payments,accounts_with_missed_payments,last_updated
0,Affluent,Very Good,Very High,Retired,High Risk,60,17.30,1.4,208.83,217112.54,58,60,2025-12-27 10:04:35.586741+00:00
1,Mass Market,Poor,High,Retired,High Risk,57,17.08,1.3,237.82,194635.64,56,57,2025-12-27 10:04:35.586741+00:00
2,Premium,Poor,High,Self-Employed,High Risk,58,16.46,1.4,194.24,197950.02,57,58,2025-12-27 10:04:35.586741+00:00
3,Mass Market,Poor,High,Self-Employed,High Risk,54,16.45,1.3,194.95,188486.10,51,54,2025-12-27 10:04:35.586741+00:00
4,Affluent,Fair,Very High,Student,High Risk,83,16.32,1.3,206.42,200594.37,83,83,2025-12-27 10:04:35.586741+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,Affluent,Poor,Very High,Retired,Low Risk,69,11.98,1.0,41.17,202193.25,46,0,2025-12-27 10:04:35.586741+00:00
110,Premium,Poor,Very High,Employed,Low Risk,81,11.96,1.0,59.29,180028.71,69,0,2025-12-27 10:04:35.586741+00:00
111,Premium,Poor,Very High,Self-Employed,Low Risk,76,11.70,1.1,54.54,207728.57,56,0,2025-12-27 10:04:35.586741+00:00
112,Affluent,Poor,Very High,Student,Low Risk,72,10.96,0.8,52.82,222956.48,52,0,2025-12-27 10:04:35.586741+00:00


## **DECLINE ANALYSIS**

### Decline Reasons Funnel

In [14]:
decline_reasons_funnel_query = """
    SELECT decline_reason, SUM(declined_count) as total_declines 
    FROM gold.analytics_transaction_decline_analysis 
    GROUP BY decline_reason 
    ORDER BY total_declines DESC
"""

decline_reasons_funnel_df = fetch_data(decline_reasons_funnel_query)

fig = go.Figure(
    go.Funnel(
        y=decline_reasons_funnel_df['decline_reason'],
        x=decline_reasons_funnel_df['total_declines'],
        textinfo="value+percent initial",
        textposition="inside",
        marker=dict(color=px.colors.sequential.Reds),
        hovertemplate='<b>%{y}</b><br>Declines: %{x:,}<extra></extra>'
    )
)

fig.update_layout(
    title=dict(
        text="Transaction Decline Reasons",
        x=0.01,
        font=dict(size=20, weight="bold")
    ),
    height=600,
    plot_bgcolor="white",
    paper_bgcolor="white",
    margin=dict(t=100, l=120, r=40, b=60)
)

fig.show()

In [15]:
decline_reasons_funnel_df

,decline_reason,total_declines
0,Invalid Card,440.0
1,Fraud Suspected,435.0
2,Insufficient Funds,408.0


### Decline By Channel Stacked

In [16]:
decline_by_channel_stacked_query = "SELECT * FROM gold.analytics_transaction_decline_analysis ORDER BY declined_count DESC"
decline_by_channel_stacked_df = fetch_data(decline_by_channel_stacked_query)

numeric_cols = decline_by_channel_stacked_df.select_dtypes(include='number').columns
channel_agg = (
    decline_by_channel_stacked_df
    .groupby('channel')[numeric_cols]
    .sum()
    .sort_values('declined_count', ascending=False)
    .reset_index()
)
top_channels_df = decline_by_channel_stacked_df[
    decline_by_channel_stacked_df['channel'].isin(channel_agg['channel'])
]

fig = px.bar(
    top_channels_df,
    x='channel',
    y='declined_count',
    color='decline_reason',
    labels={'declined_count': 'Number of Declines'},
    hover_data={
        'declined_count': ':,',
        'avg_declined_amount': ':$,.2f',
        'affected_customers': ':,'
    },
    color_discrete_sequence=px.colors.qualitative.Set2,
    title='Transaction Declines by Channel & Reason'
)

fig.update_layout(
    height=550,
    barmode='stack',
    xaxis_title='Channel',
    yaxis_title='Number of Declines',
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis=dict(tickangle=-30, showgrid=False),
    yaxis=dict(showgrid=True, gridcolor='rgba(200,200,200,0.3)'),
    margin=dict(t=100, l=80, r=40, b=100)
)

fig.show()

In [17]:
decline_by_channel_stacked_df

,decline_reason,channel,merchant_category,is_international,declined_count,pct_of_declines,avg_declined_amount,affected_customers,affected_merchants,avg_hour_of_day,avg_fraud_score,avg_merchant_risk,last_updated
0,Invalid Card,Online,Grocery,True,12,0.94,1431.54,12,12,0.0,0.149,0.40,2025-12-27 10:04:38.895277+00:00
1,Insufficient Funds,Branch,Services,True,10,0.78,1459.86,10,10,0.0,0.172,0.44,2025-12-27 10:04:38.895277+00:00
2,Insufficient Funds,POS,Retail,False,9,0.70,2435.01,9,9,0.0,0.155,0.46,2025-12-27 10:04:38.895277+00:00
3,Invalid Card,Online,Gas Station,False,9,0.70,2227.62,9,9,0.0,0.152,0.59,2025-12-27 10:04:38.895277+00:00
4,Fraud Suspected,POS,Gas Station,True,9,0.70,1603.31,9,9,0.0,0.205,0.59,2025-12-27 10:04:38.895277+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Insufficient Funds,Branch,Online Shopping,False,1,0.08,1878.60,1,1,0.0,0.151,0.09,2025-12-27 10:04:38.895277+00:00
296,Invalid Card,Branch,Retail,False,1,0.08,1119.54,1,1,0.0,0.277,0.08,2025-12-27 10:04:38.895277+00:00
297,Insufficient Funds,POS,Online Shopping,False,1,0.08,1550.16,1,1,0.0,0.030,0.96,2025-12-27 10:04:38.895277+00:00
298,Insufficient Funds,POS,Restaurant,True,1,0.08,1324.50,1,1,0.0,0.085,0.70,2025-12-27 10:04:38.895277+00:00


## **SATISFACTION ANALYSIS**

### Satisfaction Drivers Scatter 

In [18]:
satisfaction_drivers_scatter_query = "SELECT * FROM gold.analytics_customer_satisfaction_drivers WHERE interaction_count >= 20"
satisfaction_drivers_scatter_df = fetch_data(satisfaction_drivers_scatter_query)


fig = px.scatter(
    satisfaction_drivers_scatter_df,
    x='resolution_rate_pct',
    y='avg_satisfaction',
    size='interaction_count',
    color='issue_severity',
    hover_name='interaction_type',
    hover_data={
        'interaction_count': ':,',
        'resolution_rate_pct': ':.1f%',
        'avg_satisfaction': ':.2f',
        'avg_duration_minutes': ':.1f',
        'escalation_rate_pct': ':.1f%'
    },
    title='Customer Satisfaction Drivers: Resolution vs Satisfaction',
    labels={
        'resolution_rate_pct': 'Resolution Rate (%)',
        'avg_satisfaction': 'Avg Satisfaction Rating'
    },
    color_discrete_map={'Low': '#00CC96', 'Medium': '#FFA15A', 'High': '#EF553B'},
    size_max=50,
    opacity=0.8
)

# Trend line: min → max satisfaction
fig.add_trace(
    go.Scatter(
        x=[0, 100],
        y=[satisfaction_drivers_scatter_df['avg_satisfaction'].min(), satisfaction_drivers_scatter_df['avg_satisfaction'].max()],
        mode='lines',
        line=dict(dash='dash', color='gray'),
        showlegend=False,
        hoverinfo='skip'
    )
)

fig.update_layout(
    height=600,
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis=dict(showgrid=True, gridcolor='rgba(200,200,200,0.3)', range=[0, 100], ticksuffix='%'),
    yaxis=dict(showgrid=True, gridcolor='rgba(200,200,200,0.3)'),
    legend_title_text='Issue Severity',
    title=dict(x=0.01, font=dict(size=20, weight='bold')),
    margin=dict(t=100, l=80, r=40, b=80)
)

fig.show()

In [19]:
satisfaction_drivers_scatter_df

,interaction_type,reason,duration_category,issue_severity,interaction_count,avg_satisfaction,avg_duration_minutes,avg_sentiment_score,resolved_count,resolution_rate_pct,escalated_count,escalation_rate_pct,positive_count,negative_count,negative_sentiment_pct,unique_customers,last_updated
0,Phone Call,Complaint,Complex,Low,133,3.09,78.4,0.006,133,100.0,0,0.0,35,54,40.60,132,2025-12-27 10:04:33.436919+00:00
1,Phone Call,Transaction Dispute,Complex,Low,130,3.03,71.9,0.020,130,100.0,0,0.0,31,47,36.15,129,2025-12-27 10:04:33.436919+00:00
2,Branch Visit,Account Inquiry,Complex,Low,129,2.92,74.8,0.033,129,100.0,0,0.0,45,56,43.41,129,2025-12-27 10:04:33.436919+00:00
3,Phone Call,Technical Support,Complex,Medium,128,2.82,74.5,0.029,0,0.0,0,0.0,34,49,38.28,128,2025-12-27 10:04:33.436919+00:00
4,Branch Visit,Service Request,Complex,Low,128,2.97,74.7,0.064,128,100.0,0,0.0,38,47,36.72,128,2025-12-27 10:04:33.436919+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Branch Visit,Account Inquiry,Extended,Medium,20,3.33,22.2,-0.288,0,0.0,0,0.0,1,13,65.00,20,2025-12-27 10:04:33.436919+00:00
89,Email,Fraud Report,Extended,Low,20,2.80,21.8,-0.168,20,100.0,0,0.0,2,9,45.00,20,2025-12-27 10:04:33.436919+00:00
90,Branch Visit,Transaction Dispute,Complex,High,20,3.00,83.7,-0.104,11,55.0,20,100.0,5,11,55.00,20,2025-12-27 10:04:33.436919+00:00
91,Email,Technical Support,Extended,Medium,20,3.07,21.5,0.215,0,0.0,0,0.0,8,5,25.00,20,2025-12-27 10:04:33.436919+00:00


### Satisfaction By Reason Box

In [20]:
satisfaction_by_reason_box_query = "SELECT * FROM gold.analytics_customer_satisfaction_drivers WHERE interaction_count >= 10"
satisfaction_by_reason_box_df = fetch_data(satisfaction_by_reason_box_query)

fig = go.Figure()

top_reasons = (
    satisfaction_by_reason_box_df.groupby('reason')['interaction_count']
    .sum()
    .sort_values(ascending=False)
    .index
)

colors = px.colors.qualitative.Pastel

for i, reason in enumerate(top_reasons):
    reason_data = satisfaction_by_reason_box_df[satisfaction_by_reason_box_df['reason'] == reason]
    fig.add_trace(
        go.Box(
            y=reason_data['avg_satisfaction'],
            name=reason,
            boxmean='sd',
            marker_color=colors[i % len(colors)],
            hovertemplate='<b>%{fullData.name}</b><br>Satisfaction: %{y:.2f}<extra></extra>'
        )
    )

fig.update_layout(
    title=dict(
        text="Customer Satisfaction Distribution by Issue Reason (Top 10)",
        x=0.01,
        font=dict(size=20, weight="bold")
    ),
    yaxis_title="Satisfaction Rating",
    xaxis_title="Issue Reason",
    height=550,
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis=dict(tickangle=-30),
    margin=dict(t=100, l=80, r=40, b=120),
    showlegend=False
)

fig.show()

In [21]:
satisfaction_by_reason_box_df

,interaction_type,reason,duration_category,issue_severity,interaction_count,avg_satisfaction,avg_duration_minutes,avg_sentiment_score,resolved_count,resolution_rate_pct,escalated_count,escalation_rate_pct,positive_count,negative_count,negative_sentiment_pct,unique_customers,last_updated
0,Phone Call,Complaint,Complex,Low,133,3.09,78.4,0.006,133,100.0,0,0.0,35,54,40.60,132,2025-12-27 10:04:33.436919+00:00
1,Phone Call,Transaction Dispute,Complex,Low,130,3.03,71.9,0.020,130,100.0,0,0.0,31,47,36.15,129,2025-12-27 10:04:33.436919+00:00
2,Branch Visit,Account Inquiry,Complex,Low,129,2.92,74.8,0.033,129,100.0,0,0.0,45,56,43.41,129,2025-12-27 10:04:33.436919+00:00
3,Phone Call,Technical Support,Complex,Medium,128,2.82,74.5,0.029,0,0.0,0,0.0,34,49,38.28,128,2025-12-27 10:04:33.436919+00:00
4,Branch Visit,Service Request,Complex,Low,128,2.97,74.7,0.064,128,100.0,0,0.0,38,47,36.72,128,2025-12-27 10:04:33.436919+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,Branch Visit,Account Inquiry,Complex,High,10,3.29,84.6,-0.250,3,30.0,10,100.0,1,5,50.00,10,2025-12-27 10:04:33.436919+00:00
217,Branch Visit,Technical Support,Standard,Low,10,2.00,8.5,0.093,10,100.0,0,0.0,5,4,40.00,10,2025-12-27 10:04:33.436919+00:00
218,Chat,Service Request,Standard,Low,10,2.60,9.6,-0.166,10,100.0,0,0.0,2,7,70.00,10,2025-12-27 10:04:33.436919+00:00
219,Phone Call,Technical Support,Complex,High,10,3.00,76.1,0.232,6,60.0,10,100.0,4,3,30.00,10,2025-12-27 10:04:33.436919+00:00


## **CREDIT APPLICATION ANALYSIS**

### Credit Rejection Waterfall 

In [22]:
credit_rejection_waterfall_query = """
    SELECT 
        SUM(application_count) as total_apps,
        SUM(approved_count) as approved,
        SUM(application_count) - SUM(approved_count) as denied
    FROM gold.analytics_credit_application_rejection_analysis
"""

credit_rejection_waterfall_df = fetch_data(credit_rejection_waterfall_query)

total_apps = int(credit_rejection_waterfall_df['total_apps'].iloc[0])
approved = int(credit_rejection_waterfall_df['approved'].iloc[0])
denied = int(credit_rejection_waterfall_df['denied'].iloc[0])

fig = go.Figure(go.Waterfall(
    name="Applications",
    orientation="v",
    measure=["absolute", "relative", "total"],
    x=["Total Applications", "Denied", "Approved"],
    y=[total_apps, -denied, approved],
    text=[f"{total_apps:,}", f"{denied:,}", f"{approved:,}"],
    textposition="outside",
    connector={"line": {"color": "rgba(63, 63, 63, 0.7)", "width":2}},
    decreasing={"marker": {"color": "#EF553B"}},
    increasing={"marker": {"color": "#00CC96"}},
    totals={"marker": {"color": "#636EFA"}}
))

fig.update_layout(
    title=dict(
        text="Credit Application Approval Waterfall",
        x=0.01,
        font=dict(size=20, weight="bold")
    ),
    yaxis_title="Number of Applications",
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=500,
    margin=dict(t=100, l=80, r=40, b=60)
)

fig.show()

In [23]:
credit_rejection_waterfall_df

,total_apps,approved,denied
0,1847.0,446.0,1401.0


### Credit Approval Rate By Segment

In [24]:
credit_approval_rate_by_segment_query = """
    SELECT * FROM gold.analytics_credit_application_rejection_analysis 
    WHERE application_count >= 20 
    ORDER BY approval_rate_pct DESC LIMIT 15
"""

credit_approval_rate_by_segment_df = fetch_data(credit_approval_rate_by_segment_query)

credit_approval_rate_by_segment_df['segment_label'] = (
    credit_approval_rate_by_segment_df['credit_score_band'] + '<br>' +
    credit_approval_rate_by_segment_df['income_bracket'] + '<br>' +
    credit_approval_rate_by_segment_df['dti_category']
)

fig = go.Figure()

# Approved bars
fig.add_trace(
    go.Bar(
        name='Approved',
        x=credit_approval_rate_by_segment_df['segment_label'],
        y=credit_approval_rate_by_segment_df['approval_rate_pct'],
        marker_color='#00CC96',
        text=credit_approval_rate_by_segment_df['approval_rate_pct'],
        texttemplate='%{text:.1f}%',
        textposition='outside',
        hovertemplate='<b>Approved</b><br>Rate: %{y:.1f}%<extra></extra>'
    )
)

# Denied bars
fig.add_trace(
    go.Bar(
        name='Denied',
        x=credit_approval_rate_by_segment_df['segment_label'],
        y=100 - credit_approval_rate_by_segment_df['approval_rate_pct'],
        marker_color='#EF553B',
        text=100 - credit_approval_rate_by_segment_df['approval_rate_pct'],
        texttemplate='%{text:.1f}%',
        textposition='inside',
        hovertemplate='<b>Denied</b><br>Rate: %{y:.1f}%<extra></extra>'
    )
)

fig.update_layout(
    title=dict(
        text="Credit Approval Rate by Customer Segment (Top 15)",
        x=0.01,
        font=dict(size=20, weight="bold")
    ),
    xaxis_title="Segment",
    yaxis_title="Percentage (%)",
    barmode='stack',
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis=dict(tickangle=-35, showgrid=False),
    yaxis=dict(showgrid=True, gridcolor='rgba(200,200,200,0.3)'),
    height=600,
    margin=dict(t=100, l=80, r=40, b=120),
    legend_title_text='Status'
)

fig.show()

In [25]:
credit_approval_rate_by_segment_df

,decision,risk_grade,dti_category,credit_score_band,income_bracket,employment_status,product_name,application_count,pct_of_applications,avg_requested_amount,avg_credit_score,avg_dti_pct,avg_annual_income,avg_approval_probability_pct,approved_count,approval_rate_pct,avg_approved_amount,last_updated,segment_label


## **ACCOUNT CLOSURE ANALYSIS**

### Account Closure Sankey

In [26]:
account_closure_sankey_query = "SELECT * FROM gold.analytics_account_closure_analysis WHERE closed_accounts >= 10"
account_closure_sankey_df = fetch_data(account_closure_sankey_query)

sources = []
targets = []
values = []
labels_set = set()
node_colors = {}

# Define colors for each type
segment_color = "#636EFA"
product_color = "#00CC96"
risk_color = "#EF553B"

for _, row in account_closure_sankey_df.iterrows():
    segment = row['customer_segment']
    product = row['product_category']
    risk = row['churn_risk_category']

    labels_set.update([segment, product, risk])

    # Map colors
    node_colors[segment] = segment_color
    node_colors[product] = product_color
    node_colors[risk] = risk_color

    # Add links
    sources.append(segment)
    targets.append(product)
    values.append(row['closed_accounts'])

    sources.append(product)
    targets.append(risk)
    values.append(row['closed_accounts'])

labels = list(labels_set)
colors = [node_colors[label] for label in labels]

fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=25,
        label=labels,
        color=colors
    ),
    link=dict(
        source=[labels.index(s) for s in sources],
        target=[labels.index(t) for t in targets],
        value=values,
        color="rgba(150,150,150,0.4)"  # subtle link color
    )
)])

fig.update_layout(
    title=dict(
        text="Account Closure Flow: Segment → Product → Risk",
        x=0.01,
        font=dict(size=20, weight="bold")
    ),
    height=600,
    plot_bgcolor="white",
    paper_bgcolor="white",
    margin=dict(t=100, l=80, r=40, b=60)
)

fig.show()

In [27]:
account_closure_sankey_df

,product_category,customer_segment,churn_risk_category,closed_accounts,avg_account_age_months,avg_customer_tenure_months,avg_transactions_before_closure,avg_service_contacts,avg_negative_interactions,avg_monthly_fee,zero_balance_closures,negative_balance_closures,last_updated
0,DEPOSIT,Affluent,High Risk,123,19.3,16.5,0.0,0.3,0.1,4.27,0,0,2025-12-27 10:04:31.717463+00:00
1,DEPOSIT,Premium,High Risk,113,19.6,12.7,0.0,0.2,0.1,4.65,0,0,2025-12-27 10:04:31.717463+00:00
2,DEPOSIT,Business,High Risk,108,19.2,14.8,0.0,0.2,0.1,3.47,0,0,2025-12-27 10:04:31.717463+00:00
3,DEPOSIT,Mass Market,Low Risk,106,19.8,16.7,0.0,0.2,0.0,3.68,0,0,2025-12-27 10:04:31.717463+00:00
4,DEPOSIT,Affluent,Low Risk,106,20.4,17.2,0.0,0.2,0.1,4.95,0,0,2025-12-27 10:04:31.717463+00:00
5,DEPOSIT,Business,Low Risk,106,20.5,16.8,0.0,0.2,0.1,3.96,0,0,2025-12-27 10:04:31.717463+00:00
6,DEPOSIT,Mass Market,High Risk,105,19.3,16.3,0.0,0.2,0.1,3.43,0,0,2025-12-27 10:04:31.717463+00:00
7,LOAN,Premium,High Risk,104,19.6,14.7,0.0,0.2,0.1,0.00,0,104,2025-12-27 10:04:31.717463+00:00
8,LOAN,Mass Market,High Risk,96,19.2,16.8,0.0,0.3,0.1,0.00,0,96,2025-12-27 10:04:31.717463+00:00
9,LOAN,Affluent,High Risk,94,18.9,17.0,0.0,0.3,0.1,0.00,0,94,2025-12-27 10:04:31.717463+00:00
